AI Agent 一般包含以下几个关键模块：
1. 计划模块（Planning）
- 负责理解目标任务
- 拆解任务为可执行的步骤
- 生成行动计划
2. 记忆模块（Memory）
- 存储与任务相关的历史记录、上下文信息
- 支持长期记忆与短期记忆
- 提高连续对话与任务一致性
3. 工具使用模块（Tools）
- 能调用各种外部工具，如：
  - 网络搜索引擎
  - API 接口
  - 计算器、代码执行器
- 实现感知-行动闭环
4. 执行模块（Executor）
- 根据计划执行每一步操作
- 反馈执行结果并调整策略（如果需要）

# 本地文档检索RAG

## 文档解析

利用数据解析云服务llamacloud（https://cloud.llamaindex.ai/），异步的解析文档。

In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import nest_asyncio

nest_asyncio.apply()

from llama_cloud_services import LlamaParse
import os
from dotenv import load_dotenv

load_dotenv()
LLAMA_CLOUD_API_KEY = os.getenv("LLAMA_CLOUD_API_KEY")

parser = LlamaParse(
    api_key=LLAMA_CLOUD_API_KEY,  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown",  # "markdown" and "text" are available
    num_workers=3,  # if multiple files passed, split in `num_workers` API calls
    verbose=True,
    language="ch_sim",  # Optionally you can define a language, default=en
)

file_extractor = {".pdf": parser}
# 返回一个列表
documents_cloud = SimpleDirectoryReader(
    "./data", file_extractor=file_extractor
).load_data()
# 汇总所有文档
all_doc=''
for doc in documents_cloud:
    all_doc+=doc.text

Started parsing the file under job_id 45e7fcf1-5654-4c6f-afce-2d99827be4d4


## 切块

In [2]:
def split_table_if_needed(table_text, hard_max_length=512):
    """
    将超长表格按“表头+N行”的形式拆分，保留表头，确保每段不超过 hard_max_length。
    """
    lines = table_text.strip().split('\n')
    if len('\n'.join(lines)) <= hard_max_length:
        return [table_text]  # 不用拆分
    
    if len(lines) < 2 or '|' not in lines[0]:
        return [table_text]  # 非标准表格，返回原样

    header = lines[0]
    divider = lines[1]
    rows = lines[2:]
    
    chunks = []
    current_chunk = [header, divider]
    current_len = len(header) + len(divider) + 2  # 2 for newlines
    
    for row in rows:
        row_len = len(row) + 1
        if current_len + row_len > hard_max_length:
            chunks.append('\n'.join(current_chunk))
            current_chunk = [header, divider, row]
            current_len = len(header) + len(divider) + len(row) + 3
        else:
            current_chunk.append(row)
            current_len += row_len

    if len(current_chunk) > 2:
        chunks.append('\n'.join(current_chunk))

    return chunks


def enforce_hard_max_length(chunks, hard_max_length=512):
    """
    强制将 chunk 控制在 hard_max_length 长度以内。
    若超长，则按行拆分。
    """
    final_chunks = []

    for chunk in chunks:
        if len(chunk) <= hard_max_length:
            final_chunks.append(chunk)
        else:
            lines = chunk.split('\n')
            current = []
            current_len = 0
            for line in lines:
                line_len = len(line) + 1
                if current_len + line_len > hard_max_length and current:
                    final_chunks.append('\n'.join(current))
                    current = []
                    current_len = 0
                current.append(line)
                current_len += line_len
            if current:
                final_chunks.append('\n'.join(current))
    
    return final_chunks


def chunk_text(text, max_chunk_size=300, hard_max_length=512):
    """
    将文本切分成固定大小的块，保持段落和表格的完整性，并控制最大长度。
    """
    lines = text.split('\n')
    
    chunks = []
    current_chunk = []
    current_size = 0
    in_table = False
    table_content = []

    for line in lines:
        if '|' in line and not in_table:
            in_table = True
            if current_chunk:
                chunks.append('\n'.join(current_chunk))
                current_chunk = []
                current_size = 0
            table_content.append(line)
        elif in_table:
            table_content.append(line)
            if not line.strip():
                in_table = False
                table_text = '\n'.join(table_content)
                table_chunks = split_table_if_needed(table_text, hard_max_length)
                chunks.extend(table_chunks)
                table_content = []
        else:
            line_length = len(line)
            if current_size + line_length > max_chunk_size and current_chunk:
                chunks.append('\n'.join(current_chunk))
                current_chunk = []
                current_size = 0
            current_chunk.append(line)
            current_size += line_length
            if not line.strip() and current_size > max_chunk_size // 2:
                chunks.append('\n'.join(current_chunk))
                current_chunk = []
                current_size = 0

    if in_table and table_content:
        table_text = '\n'.join(table_content)
        table_chunks = split_table_if_needed(table_text, hard_max_length)
        chunks.extend(table_chunks)

    if current_chunk:
        chunks.append('\n'.join(current_chunk))

    # 确保所有 chunk 都小于 hard_max_length
    return enforce_hard_max_length(chunks, hard_max_length=hard_max_length)


# ✅ 调用
chunks = chunk_text(all_doc, max_chunk_size=300, hard_max_length=512)


## 插入Milvus数据库

利用Milvus和BGE-M3 模型进行混合搜索。BGE-M3 模型可以将文本转换为密集向量和稀疏向量。Milvus 支持在一个 Collections 中存储这两种向量，从而可以进行混合搜索，增强搜索结果的相关性。

In [3]:
# 下载bge模型 import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
from modelscope import snapshot_download
model_dir = snapshot_download('BAAI/bge-m3',
cache_dir='./', revision='master')

# BGEM3EmbeddingFunction: https://milvus.io/api-reference/pymilvus/v2.4.x/EmbeddingModels/BGEM3EmbeddingFunction/BGEM3EmbeddingFunction.md
from pymilvus import model
bge_m3_ef = model.hybrid.BGEM3EmbeddingFunction(
    model_dir,
    use_fp16=False, 
    device="cpu"
)
chunks_embeddings = bge_m3_ef(chunks)
dense_dim = chunks_embeddings["dense"][0].shape[0]

/Users/arist/miniconda3/envs/agent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pre tokenize: 100%|██████████| 4/4 [00:00<00:00, 896.41it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 4/4 [00:04<00:00,  1.05s/it]


In [4]:
print("Num of chunks:", len(chunks))
print("Sparse document dim:", chunks_embeddings["sparse"].shape)
print("Dense document dim:", len(chunks_embeddings["dense"]), chunks_embeddings["dense"][0].shape)

Num of chunks: 62
Sparse document dim: (62, 250002)
Dense document dim: 62 (1024,)


设置 Milvus Collections 和索引

In [5]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

# 连接milvus数据库
connections.connect(
    uri="./milvus.db"
)

fields = [
    # Use auto generated id as primary key，类似数据库中的主键
    FieldSchema(
        name="pk", dtype=DataType.VARCHAR, is_primary=True, auto_id=True, max_length=100
    ),
    # Store the original text to retrieve based on semantically distance
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=512),
    # 支持稀疏检索和稠密检索
    # Milvus now supports both sparse and dense vectors,
    # we can store each in a separate field to conduct hybrid search on both vectors
    FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=dense_dim),
]
schema = CollectionSchema(fields)

col_name = "hybrid_demo"
if utility.has_collection(col_name):
    Collection(col_name).drop()
col = Collection(col_name, schema, consistency_level="Strong")
# 创建索引
sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
col.create_index("sparse_vector", sparse_index)
dense_index = {"index_type": "AUTOINDEX", "metric_type": "IP"}
col.create_index("dense_vector", dense_index)
col.load()


/Users/arist/miniconda3/envs/agent/lib/python3.10/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


插入向量数据库

In [6]:
for i in range(0, len(chunks), 50):
    col.insert(
        data=[
            chunks[i : i + 50],
            chunks_embeddings["sparse"][i : i + 50],
            chunks_embeddings["dense"][i : i + 50],
        ],
        # 这一步非常关键，明确对应字段顺序（排除 auto_id 主键）
        fields=["text", "sparse_vector", "dense_vector"]
    )
print("Number of entities inserted:", col.num_entities)

Number of entities inserted: 62


## 检索

定义密集检索、稀疏检索和混合检索函数

In [7]:
from pymilvus import (
    AnnSearchRequest,
    WeightedRanker,
)

# 密集检索
def dense_search(col, query_dense_embedding, limit=10):
    search_params = {"metric_type": "IP", "params": {}}
    res = col.search(
        [query_dense_embedding],
        anns_field="dense_vector",
        limit=limit,
        output_fields=["text"],
        param=search_params,
    )[0]
    return [hit.get("text") for hit in res]

# 稀疏检索
def sparse_search(col, query_sparse_embedding, limit=10):
    search_params = {
        "metric_type": "IP",
        "params": {},
    }
    res = col.search(
        [query_sparse_embedding],
        anns_field="sparse_vector",
        limit=limit,
        output_fields=["text"],
        param=search_params,
    )[0]
    return [hit.get("text") for hit in res]

# 混合检索
def hybrid_search(
    col,
    query_dense_embedding,
    query_sparse_embedding,
    sparse_weight=1.0,
    dense_weight=1.0,
    limit=10,
):
    dense_search_params = {"metric_type": "IP", "params": {}}
    dense_req = AnnSearchRequest(
        [query_dense_embedding], "dense_vector", dense_search_params, limit=limit
    )
    sparse_search_params = {"metric_type": "IP", "params": {}}
    sparse_req = AnnSearchRequest(
        [query_sparse_embedding], "sparse_vector", sparse_search_params, limit=limit
    )
    rerank = WeightedRanker(sparse_weight, dense_weight)
    res = col.hybrid_search(
        [sparse_req, dense_req], rerank=rerank, limit=limit, output_fields=["text"]
    )[0]
    return [hit.get("text") for hit in res]


混合检索结合了密集检索和稀疏检索的优势，因此后续只采用混合检索的结果。

In [10]:
query = '什么是以人为本的座舱'
query_embeddings = bge_m3_ef([query])
dense_results = dense_search(col, query_embeddings["dense"][0])
sparse_results = sparse_search(col, query_embeddings["sparse"]._getrow(0))
hybrid_results = hybrid_search(
    col,
    query_embeddings["dense"][0],
    query_embeddings["sparse"]._getrow(0),
    sparse_weight=0.7,
    dense_weight=1.0,
)
print(dense_results[:3])
print(sparse_results[:3])
print(hybrid_results[:3])

['ES9内饰秉承"以⼈为本"的设计理念，打造宁静、舒适、科技感⼗⾜的乘坐空间。内饰设计核⼼要素包括：\n\n- 极简主义设计⻛格，减少物理按键，提供清爽整洁的视觉体验\n- 环保可持续材料应⽤，包括回收纤维⾯料和环保⽪⾰替代品\n- 人体⼯程学座椅设计，提供⻓途驾驶的舒适⽀撑\n- 多⾊环境氛围灯，可根据场景和⼼情调节⻋内氛围\n- 声学优化设计，配合主动降噪系统，打造安静驾乘环境\n', '设计团队精⼼打造的细节包括：\n\n- 封闭式前格栅设计，减少⻛阻同时彰显电动⻋特性\n- 隐藏式⻔把⼿，在保证便利性的同时提升整⻋流线型外观\n- 全LED矩阵⼤灯组，配合贯穿式光带，创造独特夜间辨识度\n- 22英⼨空⽓动⼒学轮毂，减少涡流产⽣，提⾼能源效率\n- 全景玻璃⻋顶，提供开阔视野的同时保持优雅⻋⾝⽐例\n\n# 内饰设计：以⼈为本的未来座舱\n', '星际电动在全国已建设超过5000个超级充电站，覆盖主要城市和高速公路网络。ES9车主可享受首年充电服务费减免，以及星际专属快充通道优先权。# 智能座舱\n\n# 中控与显⽰系统\n\nES9配备先进的智能座舱系统，以⽤⼾为中⼼打造沉浸式交互体验：\n\n- 55英⼨超宽曲⾯OLED液晶仪表盘与中控屏\n- 分辨率: 7680×1080像素\n- 刷新率: 120Hz\n- 亮度: 1200尼特\n- 支持HDR10+显⽰\n- 防眩光处理\n- AR增强现实抬头显⽰系统(HUD)\n- 70英⼨等效显⽰⾯积\n- 全⾊彩显⽰\n- 虚拟图像投射距离: 7.5⽶\n- 实时导航路径指引\n- 智能驾驶状态显⽰\n- 后排娱乐系统(⾼配版/性能版标配)']
['设计团队精⼼打造的细节包括：\n\n- 封闭式前格栅设计，减少⻛阻同时彰显电动⻋特性\n- 隐藏式⻔把⼿，在保证便利性的同时提升整⻋流线型外观\n- 全LED矩阵⼤灯组，配合贯穿式光带，创造独特夜间辨识度\n- 22英⼨空⽓动⼒学轮毂，减少涡流产⽣，提⾼能源效率\n- 全景玻璃⻋顶，提供开阔视野的同时保持优雅⻋⾝⽐例\n\n# 内饰设计：以⼈为本的未来座舱\n', 'ES9内饰秉承"以⼈为本"的设计理念，打造宁静、舒适、科技感⼗⾜的乘坐空间。内饰设计核⼼要素包括：\n\n- 极简主义设计⻛格，减少物理按键，提供清爽整洁的视觉体验\n- 环保可持续材料应⽤，包括

对检索结果添加序号

In [11]:
def format_list_with_markers(input_list):
    """
    Formats a list by adding numbered markers ([1], [2], etc.) to each item
    and adding a line break after each item.
    
    Args:
        input_list: A list of strings to format
        
    Returns:
        A single formatted string
    """
    result = ""
    
    for index, item in enumerate(input_list, 1):
        # Add the numbered marker and the item content
        result += f"[{index}] {item}\n"
    
    return result
formatted_references=format_list_with_markers(hybrid_results)
print(formatted_references)

[1] 设计团队精⼼打造的细节包括：

- 封闭式前格栅设计，减少⻛阻同时彰显电动⻋特性
- 隐藏式⻔把⼿，在保证便利性的同时提升整⻋流线型外观
- 全LED矩阵⼤灯组，配合贯穿式光带，创造独特夜间辨识度
- 22英⼨空⽓动⼒学轮毂，减少涡流产⽣，提⾼能源效率
- 全景玻璃⻋顶，提供开阔视野的同时保持优雅⻋⾝⽐例

# 内饰设计：以⼈为本的未来座舱

[2] ES9内饰秉承"以⼈为本"的设计理念，打造宁静、舒适、科技感⼗⾜的乘坐空间。内饰设计核⼼要素包括：

- 极简主义设计⻛格，减少物理按键，提供清爽整洁的视觉体验
- 环保可持续材料应⽤，包括回收纤维⾯料和环保⽪⾰替代品
- 人体⼯程学座椅设计，提供⻓途驾驶的舒适⽀撑
- 多⾊环境氛围灯，可根据场景和⼼情调节⻋内氛围
- 声学优化设计，配合主动降噪系统，打造安静驾乘环境

[3] 星际电动在全国已建设超过5000个超级充电站，覆盖主要城市和高速公路网络。ES9车主可享受首年充电服务费减免，以及星际专属快充通道优先权。# 智能座舱

# 中控与显⽰系统

ES9配备先进的智能座舱系统，以⽤⼾为中⼼打造沉浸式交互体验：

- 55英⼨超宽曲⾯OLED液晶仪表盘与中控屏
- 分辨率: 7680×1080像素
- 刷新率: 120Hz
- 亮度: 1200尼特
- 支持HDR10+显⽰
- 防眩光处理
- AR增强现实抬头显⽰系统(HUD)
- 70英⼨等效显⽰⾯积
- 全⾊彩显⽰
- 虚拟图像投射距离: 7.5⽶
- 实时导航路径指引
- 智能驾驶状态显⽰
- 后排娱乐系统(⾼配版/性能版标配)
[4] # 用户口碑

# 优点TOP5（根据用户反馈）：

1. 超长续航能力，减少里程焦虑
2. 智能驾驶体验出色，大幅减轻驾驶疲劳
3. 智能座舱系统直观易用
4. 加速性能强劲，驾驶乐趣十足
5. 静谧舒适的乘坐体验

# 改进建议TOP3：

1. 充电网络覆盖还可继续完善
2. 部分功能学习曲线偏陡
3. 后备箱空间可进一步优化

[5] - 0.21Cd超低风阻系数
- 主动进气格栅，根据冷却需求自动调节开合
- 平整底盘设计，减少气流湍流


# 舒适性与便利性

# 座椅系统

ES9提供豪华舒适的座椅系统，满足长途旅行需求：

- 前排座椅
- 22向电动调节
- 记忆功能
- 加热/通风/按摩
- 人体工

## 生成

定义用户prompt

In [12]:
prompt = f"""
你是一个智能助手，负责根据用户的问题和提供的参考内容生成回答。请严格按照以下要求生成回答：
1. 回答必须基于提供的参考内容。
2. 在回答中，每一块内容都必须标注引用的来源，格式为：[引用编号]。例如：[1] 表示引用自第1条参考内容。
3. 如果没有参考内容，请明确说明。如果没有参考知识，根据你自己的知识进行回答

参考内容：
{formatted_references}

用户问题：{query}
"""

print(prompt)


你是一个智能助手，负责根据用户的问题和提供的参考内容生成回答。请严格按照以下要求生成回答：
1. 回答必须基于提供的参考内容。
2. 在回答中，每一块内容都必须标注引用的来源，格式为：[引用编号]。例如：[1] 表示引用自第1条参考内容。
3. 如果没有参考内容，请明确说明。如果没有参考知识，根据你自己的知识进行回答

参考内容：
[1] 设计团队精⼼打造的细节包括：

- 封闭式前格栅设计，减少⻛阻同时彰显电动⻋特性
- 隐藏式⻔把⼿，在保证便利性的同时提升整⻋流线型外观
- 全LED矩阵⼤灯组，配合贯穿式光带，创造独特夜间辨识度
- 22英⼨空⽓动⼒学轮毂，减少涡流产⽣，提⾼能源效率
- 全景玻璃⻋顶，提供开阔视野的同时保持优雅⻋⾝⽐例

# 内饰设计：以⼈为本的未来座舱

[2] ES9内饰秉承"以⼈为本"的设计理念，打造宁静、舒适、科技感⼗⾜的乘坐空间。内饰设计核⼼要素包括：

- 极简主义设计⻛格，减少物理按键，提供清爽整洁的视觉体验
- 环保可持续材料应⽤，包括回收纤维⾯料和环保⽪⾰替代品
- 人体⼯程学座椅设计，提供⻓途驾驶的舒适⽀撑
- 多⾊环境氛围灯，可根据场景和⼼情调节⻋内氛围
- 声学优化设计，配合主动降噪系统，打造安静驾乘环境

[3] 星际电动在全国已建设超过5000个超级充电站，覆盖主要城市和高速公路网络。ES9车主可享受首年充电服务费减免，以及星际专属快充通道优先权。# 智能座舱

# 中控与显⽰系统

ES9配备先进的智能座舱系统，以⽤⼾为中⼼打造沉浸式交互体验：

- 55英⼨超宽曲⾯OLED液晶仪表盘与中控屏
- 分辨率: 7680×1080像素
- 刷新率: 120Hz
- 亮度: 1200尼特
- 支持HDR10+显⽰
- 防眩光处理
- AR增强现实抬头显⽰系统(HUD)
- 70英⼨等效显⽰⾯积
- 全⾊彩显⽰
- 虚拟图像投射距离: 7.5⽶
- 实时导航路径指引
- 智能驾驶状态显⽰
- 后排娱乐系统(⾼配版/性能版标配)
[4] # 用户口碑

# 优点TOP5（根据用户反馈）：

1. 超长续航能力，减少里程焦虑
2. 智能驾驶体验出色，大幅减轻驾驶疲劳
3. 智能座舱系统直观易用
4. 加速性能强劲，驾驶乐趣十足
5. 静谧舒适的乘坐体验

# 改进建议TOP3：

1. 充电网络覆盖还可继续完善
2. 部分功

调用gpt-4o接口，将检索到的内容添加到用户prompt中。

In [13]:
from openai import OpenAI
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client_gpt = OpenAI(api_key=OPENAI_API_KEY, base_url="https://api.openai.com/v1")

completion_gpt = client_gpt.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ]
)

print(completion_gpt.choices[0].message.content)

“以人为本的座舱”是一种设计理念，强调在汽车内饰设计中将乘客的舒适性和体验放在首位。这种设计通常体现在多个方面，包括：

1. **极简主义设计**：减少物理按键，使得车内环境更加清爽整洁，提高视觉体验[2]。
2. **环保材料的应用**：使用可回收纤维面料和环保皮革替代品，体现对可持续发展的重视[2]。
3. **人体工程学座椅设计**：为长途驾驶提供舒适支撑，以提高乘坐体验[2]。
4. **多色环境氛围灯**：车主可以根据不同场景或心情调节车内氛围，增加人性化体验[2]。
5. **声学优化设计**：结合主动降噪系统，打造一个安静的驾乘环境，从而提升乘客的舒适感[2]。

总之，以人为本的座舱设计旨在提供一个宁静、舒适且富有科技感的乘坐空间。


## 封装成RAG接口

In [16]:
def retrieval(query):
    query_embeddings = bge_m3_ef(chunks)
    hybrid_results = hybrid_search(
        col,
        query_embeddings["dense"][0],
        query_embeddings["sparse"]._getrow(0),
        sparse_weight=0.7,
        dense_weight=1.0,
    )  
    return hybrid_results

In [17]:
def rag(query):

    formatted_references=format_list_with_markers(retrieval(query))
    
    prompt = f"""
    你是一个智能助手，负责根据用户的问题和提供的参考内容生成回答。请严格按照以下要求生成回答：
    1. 回答必须基于提供的参考内容。
    2. 在回答中，每一块内容都必须标注引用的来源，格式为：[引用编号]。例如：[1] 表示引用自第1条参考内容。
    3. 如果没有参考内容，请明确说明。如果没有参考知识，根据你自己的知识进行回答
    
    参考内容：
    {formatted_references}
    
    用户问题：{query}
    """
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
    client = OpenAI(api_key=OPENAI_API_KEY, base_url="https://api.openai.com/v1")
    
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ]
    )
    
    return formatted_references,completion.choices[0].message.content

reference,result=rag('介绍一下技术规格')
print(result)

Inference Embeddings: 100%|██████████| 4/4 [00:03<00:00,  1.00it/s]


根据提供的参考内容，星昱电动ES9的技术规格涵盖多方面的创新和优势：

1. **电池技术**：ES9采用半固态电池技术，具有更高的能量密度和更快的充电速度，领先于行业[5]。

2. **智能驾驶**：搭载自主研发的Nebula芯片，智能驾驶系统表现出色，支持更高级别的自动驾驶功能[5]。

3. **数字座舱**：ES9配备55英寸超宽曲面屏幕，具有7680×1080像素的分辨率以及120Hz的刷新率，并支持HDR10+显示，提供领先的交互体验[6]。

4. **材料与工艺**：采用高强度铝合金与碳纤维复合材料，达成轻量化效果显著[5]。

5. **可持续材料**：内饰材料使用再生PET，车身可回收材料占比高，体现了环保贡献[8]。

这些技术亮点使得ES9在市场上具备较强的竞争力。


# 网络检索

网络检索代码的实现在“websearch”文件夹中，这里只调用网络检索的接口。

In [18]:
from websearch.src.retrieval import web_retrieval
def web_search_answer(query):

    web_reference=web_retrieval(query)
    
    prompt = f"""
    
    Web 搜索结果：
    {web_reference}
    
    指令：你是一个/一名销售助手。使用提供的网络搜索结果，对给定的查询写一个全面而详细的回复。
    确保在引用后使用 [number] 标记引用。
    在回答的最后，列出带索引的相应参考文献，每个参考文献包含网络搜索结果中的网址和引用句子，按照您在上面标记的顺序排列，这些句子应该与网络搜索结果中的完全相同。
    以下是参考文献的示例：
        [1] 网址：https://www.pocketgamer.biz/news/81670/tencent-and-netease-dominated-among-chinas-top-developers-in-q1/
            引用句子：腾讯在本季度占据了国内市场收入的大约50%，相比之下2022年第一季度为40%。
    
    查询：{query}

    请你过滤掉检索结果中的不相关性的部分进行回答，如果没有相关的部分，按照你自己的知识进行回答
    """
    client = OpenAI(api_key=OPENAI_API_KEY, base_url="https://api.openai.com/v1")

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ]
    )

    return completion.choices[0].message.content
    
query="小米SU7的发动机"
print(web_search_answer(query))

Starting web crawler thread 0Starting web crawler thread 1

Starting web crawler thread 2
Starting web crawler thread 3
Starting web crawler thread 4
Starting web crawler thread 5
Starting web crawler thread 6
Starting web crawler thread 7
Starting web crawler thread 8
Starting web crawler thread 9
Thread 8 completed! Time consumed: 0.60s
Thread 2 completed! Time consumed: 0.61s
Thread 7 completed! Time consumed: 1.26s
Thread 0 completed! Time consumed: 1.71s
Thread 5 completed! Time consumed: 1.85s
Thread 3 completed! Time consumed: 2.02s
Thread 4 completed! Time consumed: 2.24s
Thread 1 completed! Time consumed: 2.29s
Thread 6 completed! Time consumed: 2.52s
Thread 9 completed! Time consumed: 64.74s


/Users/arist/Documents/rag-agent/websearch/src/retrieval.py:35: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = SentenceTransformerEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/Users/arist/miniconda3/envs/agent/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/Users/arist/Documents/rag-agent/websearch

小米SU7 Ultra 是一款电动车，并不使用传统的内燃机。根据最近的消息，这款车的特点之一是其模拟运动声浪功能，可以模拟不同的声音效，包括模仿发动机的超音浪声音和未来感的超脉冲声音。它支持通过一个40W扬声器来外放这些声浪，旨在提供更“饱和真实的效果”【1】【2】。

小米SU7 Ultra 的电动设计使其在性能和环保方面具有优势，作为电动车，其动力系统与传统汽车的发动机有着显著不同。电动汽车通常通过电动机进行驱动，并提供即时的扭矩和快速加速，同时具备较低的维护成本。关于电动机的具体技术细节和性能指标，目前的信息中并未详细列出。

雷军作为小米的创始人和CEO，对于SU7 Ultra的声浪功能表现出很高的关注，他在社交平台上也对此进行了推广【1】。这表明小米在开发这款车时，注重不仅仅是车辆的行驶性能，同时也在意用户的驾驶体验和情感共鸣。

如果您需要更多关于小米SU7 Ultra的具体技术规格或驾驶体验的资料，请随时告诉我，我将很乐意为您提供帮助。

参考文献：
[1] 网址：https://i.ifeng.com/c/8goI3JSUjEJ
引用句子：“新车可选多种声浪音效，包括模仿发动机的超音浪声音和具有未来感的超脉冲声音。此外，该车支持 40W 扬声器外放声浪，具有‘饱和真实的效果’。”  
[2] 网址：https://i.ifeng.com/c/8goI3JSUjEJ
引用句子：“随后，小米创办人、董事长兼 CEO 雷军转发这条微博并配文：‘这个声浪不错吧？’”


# Agent

## planning模块

定义planning模块，根据用户问题，决定使用哪个工具来查询以获得更多需要的信息（本地文档搜索或网络搜索）

In [19]:
from openai import OpenAI
import os

def middle_json_model(prompt):

    client = OpenAI(
        api_key=OPENAI_API_KEY, 
        base_url="https://api.openai.com/v1",
    )
    completion = client.chat.completions.create(
        model="gpt-4o-mini", # 此处以gpt-4o-mini为例，可按需更换模型名称。国产模型可以参考列表：https://help.aliyun.com/zh/model-studio/getting-started/models
        messages=[
            {'role': 'system', 'content': 'You are a helpful assistant.'},
            {'role': 'user', 'content': prompt}],
        response_format={"type": "json_object"}
        )
        
    return completion.choices[0].message.content

提取有效的json输出

In [20]:
import re
import json

def extract_json_content(input_str):
    """
    提取字符串中第一个"["和最后一个"]"之间的内容（包括中括号）
    
    Args:
        input_str (str): 需要处理的输入字符串
    
    Returns:
        str or None: 提取的JSON内容，如果没有匹配则返回None
    """
    # 使用正则表达式匹配第一个"["到最后一个"]"之间的内容
    # [\s\S]* 匹配任意字符（包括换行符）
    pattern = r'(\[[\s\S]*\])'
    match = re.search(pattern, input_str)
    
    # 如果匹配成功，返回匹配的内容；否则返回None
    return match.group(1) if match else None

测试planning模块

In [23]:
#规划模块plan
import json

def agent_plan(query):
    prompt='''

你是一个专业的汽车销售助手的规划模块。你的任务是：
1. 分析用户的查询:{0}
2. 基于已有的信息，决定使用哪个工具来查询以获得更多需要的信息（本地文档搜索或网络搜索）
3. 将用户的原始查询拆解或延伸为1-2个相关问题，以获取更全面的信息


## 可用工具
1. **本地文档搜索**：搜索本地星辰电动ES9的文档，包含以下章节：
   - 产品概述
   - 设计理念
   - 技术规格
   - 驱动系统
   - 电池与充电
   - 智能座舱
   - 智能驾驶
   - 安全系统
   - 车身结构
   - 舒适性与便利性
   - 版本与配置
   - 价格与购买信息
   - 售后服务
   - 环保贡献
   - 用户评价
   - 竞品对比
   - 常见问题
   - 联系方式

2. **网络搜索**：在互联网上搜索相关信息

## 工具选择规则
- 当查询明确涉及星辰电动ES9的具体信息、参数、功能或服务时，优先使用**本地文档搜索**
- 当查询涉及以下情况时，使用**网络搜索**：
  - 与其他品牌车型的详细对比
  - 最新市场动态或新闻
  - 非官方的用户体验或评测
  - 星辰电动ES9文档中可能没有的信息
  - 需要实时数据（如当前市场价格波动等）

## prompt延伸的规则
- 本地检索的查询扩展侧重于产品信息的深度查询
- 网络检索的查询扩展侧重于本地无法检索到的信息

## 输出格式
你的输出应该是一个JSON格式的列表，每个项目包含：
1. `action_name`：工具名称（"本地文档搜索"或"网络搜索"）
2. `prompts`：问题列表，第一个是原始查询，后面是拆解或延伸的问题
[
  {{
    "action_name": "工具名称",
    "prompts": [
      "原始查询",
      "拆解/延伸问题1",
      "拆解/延伸问题2",
      "拆解/延伸问题3"
    ]
  }}
]


## 示例

### 示例1：关于车辆规格的查询
用户：星辰电动ES9的续航里程是多少？

输出：
[
  {{
    "action_name": "本地文档搜索",
    "prompts": [
      "星辰电动ES9的续航里程是多少？",
      "星辰电动ES9的电池容量是多少？",
      "星辰电动ES9不同版本的续航里程有何区别？"
    ]
  }}
]


### 示例2：关于市场比较的查询
用户：星辰电动ES9和特斯拉Model Y相比怎么样？

输出：
[
  {{
    "action_name": "本地文档搜索",
    "prompts": [
      "星辰电动ES9的主要优势和特点是什么？",
      "星辰电动ES9的技术规格和配置有哪些？"
    ]
  }},
  {{
    "action_name": "网络搜索",
    "prompts": [
      "特斯拉Model Y主要优势和特点？",
      "特斯拉Model Y最新规格和价格",
      "特斯拉Model Y技术规格和配置有哪些"
    ]
  }}
]


### 示例3：关于日常问题
用户：你好
这种情况下都不需要调用，则输出为None

只需要输出JSON的部分，前后不要输出任何信息

'''.format(query)
    result=(middle_json_model(prompt))
    # print(result)
    # json_list=extract_json_content(result) # not useful here
    try:
        structure_output=json.loads(result)
    except:
        structure_output = None

    return structure_output
    
user_query='比较一下和华为汽车的优劣势'
action_tool=agent_plan(user_query)
print(action_tool)

{'action_name': '本地文档搜索', 'prompts': ['比较星辰电动ES9和华为汽车的优劣势是什么？', '星辰电动ES9的特性与华为汽车的技术参数有哪些具体区别？', '星辰电动ES9在智能驾驶和安全系统方面与华为汽车相比如何？']}


调整数据格式，使每个action_name只搭配一个prompt

In [24]:
#任务状态state
def adjust_format(original_data):
    """
    调整数据格式，使每个action_name只搭配一个prompt
    
    参数:
    original_data (list): 原始数据，每个action_name对应多个prompts
    
    返回:
    list: 调整后的数据，每个action_name只对应一个prompt
    """
    adjusted_data = []

    for item in original_data:
        action_name = item['action_name']
        prompts = item['prompts']
        
        # 为每个prompt创建一个新的字典
        for prompt in prompts:
            adjusted_item = {
                'action_name': action_name,
                'prompt': prompt
            }
            adjusted_data.append(adjusted_item)
    
    return adjusted_data

if action_tool:
    if isinstance(action_tool, dict):
        action_tool = [action_tool]
    adjusted_tools = adjust_format(action_tool)
    actions = adjusted_tools
else:
    actions=[]
print(actions)

[{'action_name': '本地文档搜索', 'prompt': '比较星辰电动ES9和华为汽车的优劣势是什么？'}, {'action_name': '本地文档搜索', 'prompt': '星辰电动ES9的特性与华为汽车的技术参数有哪些具体区别？'}, {'action_name': '本地文档搜索', 'prompt': '星辰电动ES9在智能驾驶和安全系统方面与华为汽车相比如何？'}]


## tools模块

执行模块，依次执行actions内的动作，根据action_name判断执行函数web_search_answer()还是rag()，web_search_answer()和rag()就相当于tools，可以供agent使用

In [ ]:
def process_actions(actions):
    """
    处理动作列表函数
    
    Args:
        actions: 动作列表，每个动作包含action_name和prompt
        
    Returns:
        memory: 包含每次调用结果的记忆列表
    """
    # 初始化记忆列表
    memory = []
    
    # 依次处理每个动作
    for action in actions:
        action_name = action['action_name']
        prompt = action['prompt']
        
        print(f'正在执行{action_name}: "{prompt}"')
        
        try:
            # 根据动作类型调用相应的函数
            if action_name == '本地文档搜索':
                result = rag(prompt)
            elif action_name == '网络搜索':
                result = web_search_answer(prompt)
            else:
                result = f"未知的动作类型: {action_name}"
            
            # 将结果添加到记忆中
            memory_item = {
                "提问": prompt,
                "结果": result
            }
            memory.append(memory_item)
            
            # 输出结果
            print(f"提问：{prompt}")
            print(f"结果：{result}")
            print("-------------------")
            
        except Exception:
            # 如果执行失败，静默处理，继续下一轮循环
            print("-------------------")
            continue
    
    print("所有执行动作已成功完成，结果已返回。")
    return memory

## memory模块

记忆模块memory，用来储存搜索到的信息

In [26]:
memory_global=[]

In [27]:
new_memory = process_actions(actions)

正在执行本地文档搜索: "比较星辰电动ES9和华为汽车的优劣势是什么？"


Inference Embeddings: 100%|██████████| 4/4 [00:04<00:00,  1.01s/it]


提问：比较星辰电动ES9和华为汽车的优劣势是什么？
结果：('[1] 星⾠电动ES9·未来旗舰电动SUV产品介绍\n# ⽬录\n\n1. 产品概述\n2. 设计理念\n3. 技术规格\n4. 驱动系统\n5. 电池与充电\n6. 智能座舱\n7. 智能驾驶\n8. 安全系统\n9. ⻋⾝结构\n10. 舒适性与便利性\n11. 版本与配置\n12. 价格与购买信息\n13. 售后服务\n14. 环保贡献\n15. ⽤⼾评价\n16. 竞品对⽐\n17. 常⻅问题\n18. 联系⽅式\n\n[2] # 产品概述\n\n星⾠电动ES9是星⾠汽⻋公司推出的旗舰级纯电动智能SUV，代表了当前电动汽⻋领域的最⾼技术⽔平。ES9采⽤全新第四代电动平台架构，集成了前沿电池技术、智能驾驶辅助系统和创新交互体验，为⽤⼾提供安全、⾼效、舒适的出⾏解决⽅案。\n\nES9以"科技融⼊⽣活，电动改变未来"为产品理念，通过⽆缝连接⻋辆、⽤⼾和环境，创造全⽅位的智能移动体验。⽆论是⽇常通勤、⻓途旅⾏还是家庭出游，ES9都能满⾜现代⽤⼾多样化的⽤⻋需求。\n\n[3] 作为星⾠汽⻋的旗舰产品，ES9凝聚了公司⼗年电动汽⻋研发经验，是智能电动出⾏的典范之作。\n\n\n# 设计理念\n\n# 外观设计：未来主义与空⽓动⼒学的完美融合\n\nES9的外观设计由国际知名设计师团队主导，采⽤"流体动⼒学"设计语⾔，将未来感与实⽤性完美结合。⻋⾝线条流畅优雅，同时实现了0.21的超低⻛阻系数，处于全球SUV领先⽔平。\n\n[4] # ⽤⼾评价\n\n# 专业媒体评测\n\n# 《电动汽⻋世界》\n\n★★★★★\n\n"ES9代表了当前电动SUV的最⾼⽔平，尤其在智能驾驶与电池技术⽅⾯，处于绝对领先地位。"\n\n# 《汽⻋测评》\n\n★★★★☆\n\n"出⾊的驾驶感受与豪华感，智能座舱体验堪称⼀流，唯⼀的缺点是价格偏⾼。"\n\n# 《科技出⾏》\n\n★★★★★\n\n"从科技配置到驾驶体验，ES9都向我们展⽰了电动汽⻋的未来形态，尤其是其智能驾驶系统令⼈印象深刻。"\n\n[5] # 优势分析\n\n星⾠ES9相⽐竞品具有以下显著优势：\n\n1. 电池技术：半固态电池技术领先⾏业，能量密度更⾼，充电速度更快\n2. 智能驾驶：⾃研Nebula芯⽚算⼒⾏业领先，⽀持更⾼级别⾃动驾驶功能\n3. 

Inference Embeddings: 100%|██████████| 4/4 [00:03<00:00,  1.03it/s]


提问：星辰电动ES9的特性与华为汽车的技术参数有哪些具体区别？
结果：('[1] 星⾠电动ES9·未来旗舰电动SUV产品介绍\n# ⽬录\n\n1. 产品概述\n2. 设计理念\n3. 技术规格\n4. 驱动系统\n5. 电池与充电\n6. 智能座舱\n7. 智能驾驶\n8. 安全系统\n9. ⻋⾝结构\n10. 舒适性与便利性\n11. 版本与配置\n12. 价格与购买信息\n13. 售后服务\n14. 环保贡献\n15. ⽤⼾评价\n16. 竞品对⽐\n17. 常⻅问题\n18. 联系⽅式\n\n[2] # 产品概述\n\n星⾠电动ES9是星⾠汽⻋公司推出的旗舰级纯电动智能SUV，代表了当前电动汽⻋领域的最⾼技术⽔平。ES9采⽤全新第四代电动平台架构，集成了前沿电池技术、智能驾驶辅助系统和创新交互体验，为⽤⼾提供安全、⾼效、舒适的出⾏解决⽅案。\n\nES9以"科技融⼊⽣活，电动改变未来"为产品理念，通过⽆缝连接⻋辆、⽤⼾和环境，创造全⽅位的智能移动体验。⽆论是⽇常通勤、⻓途旅⾏还是家庭出游，ES9都能满⾜现代⽤⼾多样化的⽤⻋需求。\n\n[3] 作为星⾠汽⻋的旗舰产品，ES9凝聚了公司⼗年电动汽⻋研发经验，是智能电动出⾏的典范之作。\n\n\n# 设计理念\n\n# 外观设计：未来主义与空⽓动⼒学的完美融合\n\nES9的外观设计由国际知名设计师团队主导，采⽤"流体动⼒学"设计语⾔，将未来感与实⽤性完美结合。⻋⾝线条流畅优雅，同时实现了0.21的超低⻛阻系数，处于全球SUV领先⽔平。\n\n[4] # ⽤⼾评价\n\n# 专业媒体评测\n\n# 《电动汽⻋世界》\n\n★★★★★\n\n"ES9代表了当前电动SUV的最⾼⽔平，尤其在智能驾驶与电池技术⽅⾯，处于绝对领先地位。"\n\n# 《汽⻋测评》\n\n★★★★☆\n\n"出⾊的驾驶感受与豪华感，智能座舱体验堪称⼀流，唯⼀的缺点是价格偏⾼。"\n\n# 《科技出⾏》\n\n★★★★★\n\n"从科技配置到驾驶体验，ES9都向我们展⽰了电动汽⻋的未来形态，尤其是其智能驾驶系统令⼈印象深刻。"\n\n[5] # 优势分析\n\n星⾠ES9相⽐竞品具有以下显著优势：\n\n1. 电池技术：半固态电池技术领先⾏业，能量密度更⾼，充电速度更快\n2. 智能驾驶：⾃研Nebula芯⽚算⼒⾏业领先，⽀持更⾼级别⾃动驾驶功

Inference Embeddings: 100%|██████████| 4/4 [00:03<00:00,  1.02it/s]


提问：星辰电动ES9在智能驾驶和安全系统方面与华为汽车相比如何？
结果：('[1] 星⾠电动ES9·未来旗舰电动SUV产品介绍\n# ⽬录\n\n1. 产品概述\n2. 设计理念\n3. 技术规格\n4. 驱动系统\n5. 电池与充电\n6. 智能座舱\n7. 智能驾驶\n8. 安全系统\n9. ⻋⾝结构\n10. 舒适性与便利性\n11. 版本与配置\n12. 价格与购买信息\n13. 售后服务\n14. 环保贡献\n15. ⽤⼾评价\n16. 竞品对⽐\n17. 常⻅问题\n18. 联系⽅式\n\n[2] # 产品概述\n\n星⾠电动ES9是星⾠汽⻋公司推出的旗舰级纯电动智能SUV，代表了当前电动汽⻋领域的最⾼技术⽔平。ES9采⽤全新第四代电动平台架构，集成了前沿电池技术、智能驾驶辅助系统和创新交互体验，为⽤⼾提供安全、⾼效、舒适的出⾏解决⽅案。\n\nES9以"科技融⼊⽣活，电动改变未来"为产品理念，通过⽆缝连接⻋辆、⽤⼾和环境，创造全⽅位的智能移动体验。⽆论是⽇常通勤、⻓途旅⾏还是家庭出游，ES9都能满⾜现代⽤⼾多样化的⽤⻋需求。\n\n[3] 作为星⾠汽⻋的旗舰产品，ES9凝聚了公司⼗年电动汽⻋研发经验，是智能电动出⾏的典范之作。\n\n\n# 设计理念\n\n# 外观设计：未来主义与空⽓动⼒学的完美融合\n\nES9的外观设计由国际知名设计师团队主导，采⽤"流体动⼒学"设计语⾔，将未来感与实⽤性完美结合。⻋⾝线条流畅优雅，同时实现了0.21的超低⻛阻系数，处于全球SUV领先⽔平。\n\n[4] # ⽤⼾评价\n\n# 专业媒体评测\n\n# 《电动汽⻋世界》\n\n★★★★★\n\n"ES9代表了当前电动SUV的最⾼⽔平，尤其在智能驾驶与电池技术⽅⾯，处于绝对领先地位。"\n\n# 《汽⻋测评》\n\n★★★★☆\n\n"出⾊的驾驶感受与豪华感，智能座舱体验堪称⼀流，唯⼀的缺点是价格偏⾼。"\n\n# 《科技出⾏》\n\n★★★★★\n\n"从科技配置到驾驶体验，ES9都向我们展⽰了电动汽⻋的未来形态，尤其是其智能驾驶系统令⼈印象深刻。"\n\n[5] # 优势分析\n\n星⾠ES9相⽐竞品具有以下显著优势：\n\n1. 电池技术：半固态电池技术领先⾏业，能量密度更⾼，充电速度更快\n2. 智能驾驶：⾃研Nebula芯⽚算⼒⾏业领先，⽀持更⾼级别⾃动驾驶

In [34]:
new_memory[1:]

[{'提问': '星辰电动ES9的特性与华为汽车的技术参数有哪些具体区别？',
  '结果': ('[1] 星⾠电动ES9·未来旗舰电动SUV产品介绍\n# ⽬录\n\n1. 产品概述\n2. 设计理念\n3. 技术规格\n4. 驱动系统\n5. 电池与充电\n6. 智能座舱\n7. 智能驾驶\n8. 安全系统\n9. ⻋⾝结构\n10. 舒适性与便利性\n11. 版本与配置\n12. 价格与购买信息\n13. 售后服务\n14. 环保贡献\n15. ⽤⼾评价\n16. 竞品对⽐\n17. 常⻅问题\n18. 联系⽅式\n\n[2] # 产品概述\n\n星⾠电动ES9是星⾠汽⻋公司推出的旗舰级纯电动智能SUV，代表了当前电动汽⻋领域的最⾼技术⽔平。ES9采⽤全新第四代电动平台架构，集成了前沿电池技术、智能驾驶辅助系统和创新交互体验，为⽤⼾提供安全、⾼效、舒适的出⾏解决⽅案。\n\nES9以"科技融⼊⽣活，电动改变未来"为产品理念，通过⽆缝连接⻋辆、⽤⼾和环境，创造全⽅位的智能移动体验。⽆论是⽇常通勤、⻓途旅⾏还是家庭出游，ES9都能满⾜现代⽤⼾多样化的⽤⻋需求。\n\n[3] 作为星⾠汽⻋的旗舰产品，ES9凝聚了公司⼗年电动汽⻋研发经验，是智能电动出⾏的典范之作。\n\n\n# 设计理念\n\n# 外观设计：未来主义与空⽓动⼒学的完美融合\n\nES9的外观设计由国际知名设计师团队主导，采⽤"流体动⼒学"设计语⾔，将未来感与实⽤性完美结合。⻋⾝线条流畅优雅，同时实现了0.21的超低⻛阻系数，处于全球SUV领先⽔平。\n\n[4] # ⽤⼾评价\n\n# 专业媒体评测\n\n# 《电动汽⻋世界》\n\n★★★★★\n\n"ES9代表了当前电动SUV的最⾼⽔平，尤其在智能驾驶与电池技术⽅⾯，处于绝对领先地位。"\n\n# 《汽⻋测评》\n\n★★★★☆\n\n"出⾊的驾驶感受与豪华感，智能座舱体验堪称⼀流，唯⼀的缺点是价格偏⾼。"\n\n# 《科技出⾏》\n\n★★★★★\n\n"从科技配置到驾驶体验，ES9都向我们展⽰了电动汽⻋的未来形态，尤其是其智能驾驶系统令⼈印象深刻。"\n\n[5] # 优势分析\n\n星⾠ES9相⽐竞品具有以下显著优势：\n\n1. 电池技术：半固态电池技术领先⾏业，能量密度更⾼，充电速度更快\n2. 智能驾驶：⾃研Nebula芯⽚算⼒⾏业领

In [67]:
memory_global.extend(new_memory[1:])

## 封装成agent接口

调用deepseek r1接口，将检索到的内容添加到用户prompt中，生成包含思考和推理两部分

In [ ]:
import os
from openai import OpenAI

# 初始化OpenAI客户端
def final_answer(memory_global,user_query):

    # DeepSeek-R1: https://api-docs.deepseek.com/zh-cn/guides/reasoning_model
    DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
    client_deepseek = OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com")
    
    reasoning_content = ""  # 定义完整思考过程
    answer_content = ""     # 定义完整回复
    is_answering = False   # 判断是否结束思考过程并开始回复
    
    
    final_prompt=f'''
        你是一个星辰电动ES9的智能销售助手，负责根据用户的问题和提供的参考内容生成回答。请严格按照以下要求生成回答：
        基于提供的参考内容进行回答，如果原文没有参考内容,根据你自己的知识进行回答
        你需要用有打动力的销售的语言进行输出，突出星辰电动的优势
        
        参考内容：
        {memory_global}
        
        用户问题：{user_query}
    
    '''
    print(final_prompt)    
    print('-'*130)
    
    # 创建聊天完成请求
    completion = client_deepseek.chat.completions.create(
        model="deepseek-reasoner",  # 此处以 deepseek-r1 为例，可按需更换模型名称
        messages=[
            {"role": "user", "content": final_prompt}
        ],
        stream=True,
        # 解除以下注释会在最后一个chunk返回Token使用量
        stream_options={
            "include_usage": True
        }
    )
    
    print("\n" + "=" * 20 + "思考过程" + "=" * 20 + "\n")

    for chunk in completion:
        # 如果chunk.choices为空，则打印usage
        if not chunk.choices:
            print("\nUsage:")
            print(chunk.usage)
        else:
            delta = chunk.choices[0].delta
            # 打印思考过程
            if hasattr(delta, 'reasoning_content') and delta.reasoning_content != None:
                print(delta.reasoning_content, end='', flush=True)
                reasoning_content += delta.reasoning_content
            else:
                # 开始回复
                content = delta.content or ""
                if content and not is_answering:
                    print("\n" + "=" * 20 + "完整回复" + "=" * 20 + "\n")
                    is_answering = True
                # 打印回复过程
                print(content, end='', flush=True)
                answer_content += delta.content
    
    print("=" * 20 + "完整思考过程" + "=" * 20 + "\n")
    print(reasoning_content)
    print("=" * 20 + "完整回复" + "=" * 20 + "\n")
    print(answer_content)

封装成agent接口，其中agent_plan、process_actions、final_answer可以视为三个子agent，协同工作。

In [55]:
def agent(user_query):
    #记忆模块
    memory_global=[]
    #获得执行清单
    action_tool=agent_plan(user_query)
    if action_tool:
        if isinstance(action_tool, dict):
            action_tool = [action_tool]
        adjusted_tools = adjust_format(action_tool)
        actions = adjusted_tools
    else:
        actions=[]
    if actions:
        #进行任务执行
        new_memory=process_actions(actions)
        memory_global.extend(new_memory[1:])
    final_answer(memory_global, user_query)

user_query='比较一下和华为汽车的优劣势'
agent(user_query)

正在执行网络搜索: "比较星辰电动ES9和华为汽车的优劣势。"


/Users/arist/miniconda3/envs/agent/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


-------------------
正在执行网络搜索: "华为汽车的主要优势和特点是什么？"
Starting web crawler thread 0Starting web crawler thread 1

Starting web crawler thread 2
Thread 2 completed! Time consumed: 0.00s
Starting web crawler thread 3
Starting web crawler thread 4
Starting web crawler thread 5
Starting web crawler thread 6
Starting web crawler thread 7
Starting web crawler thread 8
Starting web crawler thread 9
Thread 4 completed! Time consumed: 0.68s
Thread 8 completed! Time consumed: 1.09s
Thread 1 completed! Time consumed: 1.48s
Thread 6 completed! Time consumed: 1.80s
Thread 3 completed! Time consumed: 1.83s
Thread 9 completed! Time consumed: 1.85s
Thread 0 completed! Time consumed: 1.96s
Thread 5 completed! Time consumed: 1.99s
Thread 7 completed! Time consumed: 4.84s
提问：华为汽车的主要优势和特点是什么？
结果：华为在汽车领域的主要优势和特点主要体现在以下几个方面：

1. **技术创新与研发实力**：华为对技术创新的重视表现在其高额的研发投入上。到2022年底，华为的研发员工已超过11.4万人，占总员工数的55.4%。华为汽车业务的研发团队规模达到7000人，其中智能驾驶团队接近5000人，并且在2022年，华为智能汽车解决方案的研发投入累计达到30亿美元。这些举措使华为能够在技术上保持领先地位，并持续推动汽车智能化和网联化的发展。

2

/Users/arist/miniconda3/envs/agent/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


-------------------
所有执行动作已完成，结果已返回。

        你是一个星辰电动ES9的智能销售助手，负责根据用户的问题和提供的参考内容生成回答。请严格按照以下要求生成回答：
        基于提供的参考内容进行回答，如果原文没有参考内容,根据你自己的知识进行回答
        你需要用有打动力的销售的语言进行输出，突出星辰电动的优势
        
        参考内容：
        []
        
        用户问题：比较一下和华为汽车的优劣势
    
    
----------------------------------------------------------------------------------------------------------------------------------

====================思考过程====================

嗯，用户让我比较星辰电动ES9和华为汽车的优劣势，但提供的参考内容居然是空的。这就有点棘手了，因为作为销售助手，按理应该基于参考资料回答，可资料里什么信息都没有。

用户可能是正在对比不同品牌的电动车，想看看ES9的竞争力在哪里。ta提到华为汽车，说明对科技属性比较关注，毕竟华为在智能驾驶领域名声很响。不过ta没具体说华为哪款车，可能是问界系列？这种开放式比较问题其实挺考验销售话术的。

既然没有参考资料，只能从我对ES9的基础认知来回应了。重点要突出星辰的核心优势——比如三电技术这种硬实力，还有家庭用户看重的空间舒适性。华为强项在智能座舱，但星辰的智能驾驶也不弱，得强调实际体验的流畅性。

用户可能隐含担心新势力品牌的技术积累，所以回复里要特别提星辰的制造底蕴。价格方面虽然不确定，但可以用"价值比"这种话术绕开具体数字。最后得用场景化描述打动ta，比如全家出游的画面，毕竟买中大型SUV的多是家庭用户。

对了，结尾必须主动邀约试驾，销售话术的黄金法则。语气要自信但不能拉踩友商，说"不同赛道"就显得大气。
====================完整回复====================

好的！作为星辰电动ES9的销售专家，我非常理解您希望了解我们与华为汽车的对比。虽然具体细节各有侧重，但

## 实现能自我反思的agent

在prompt中要求agent自我反思，基于已有的信息，是否还需要延伸再进行查询

In [ ]:
def reflection(user_query, memory_global):
    prompt='''
    你是一个专业的汽车销售助手的规划模块。你的任务是：
1. 分析用户的查询:{0}
2. 基于已有的信息，是否还需要延伸再进行查询

##目前已有的信息:
{1}


## 可用工具
1. **本地文档搜索**：搜索本地星辰电动ES9的文档，包含以下章节：
   - 产品概述
   - 设计理念
   - 技术规格
   - 驱动系统
   - 电池与充电
   - 智能座舱
   - 智能驾驶
   - 安全系统
   - 车身结构
   - 舒适性与便利性
   - 版本与配置
   - 价格与购买信息
   - 售后服务
   - 环保贡献
   - 用户评价
   - 竞品对比
   - 常见问题
   - 联系方式

2. **网络搜索**：在互联网上搜索相关信息

## 工具选择规则
- 当查询明确涉及星辰电动ES9的具体信息、参数、功能或服务时，优先使用**本地文档搜索**
- 当查询涉及以下情况时，使用**网络搜索**：
  - 与其他品牌车型的详细对比
  - 最新市场动态或新闻
  - 非官方的用户体验或评测
  - 星辰电动ES9文档中可能没有的信息
  - 需要实时数据（如当前市场价格波动等）

## prompt延伸的规则
- 本地检索的查询扩展侧重于产品信息的深度查询
- 网络检索的查询扩展侧重于本地无法检索到的信息

###重要！
至多再扩展不超过3个查询，如果需要扩展则按照下面的输出格式输出，如果不需要则返回None




## 输出格式
你的输出应该是一个JSON格式的列表，每个项目包含：
1. `action_name`：工具名称（"本地文档搜索"或"网络搜索"）
2. `prompts`：一个扩展的问题，如果是网络检索，prompt不包含电动ES9，如果是本地检索，prompt只包含询问电动ES9，检索内容一定是一个简单问题，不包含对比
[
  {{
    "action_name": "工具名称",
    "prompts":'查询内容'
  }}
  ...
]

    '''.format(user_query, memory_global)
    result=(middle_json_model(prompt))
    # print(result)
    json_list=extract_json_content(result)
    try:
        structure_output=json.loads(json_list)
    except:
        structure_output = None

    return structure_output
        
print(reflection(user_query,memory_global))

[{'action_name': '网络搜索', 'prompts': '华为汽车的优势和劣势是什么？'}, {'action_name': '网络搜索', 'prompts': '华为汽车在市场上的竞争力如何？'}, {'action_name': '网络搜索', 'prompts': '用户对华为汽车的评价和体验如何？'}]


封装成agent接口，在执行完process_actions后，agent进行反思，直到检索到的内容足够完善

In [57]:
def agent_reflection(user_query):
    memory_global=[]
    print("开始任务规划...")
    action_tool=agent_plan(user_query)
    if action_tool:
        if isinstance(action_tool, dict):
            action_tool = [action_tool]
        adjusted_tools = adjust_format(action_tool)
        actions = adjusted_tools
    else:
        actions=[]
    if actions:
        print("开始任务执行...")
        memory_new=process_actions(actions)
        memory_global.extend(memory_new[1:])
    action_reflect=reflection(user_query,memory_global)
    if action_reflect:
        print("回顾内容，进行反思...")
        memory_new=process_actions(actions)
        memory_global.extend(memory_new)
    final_answer(memory_global,user_query)
print(agent_reflection(user_query))

开始任务规划...
开始任务执行...
正在执行网络搜索: "星辰电动ES9与华为汽车相比有哪些优劣势？"


/Users/arist/miniconda3/envs/agent/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


-------------------
正在执行网络搜索: "华为汽车的主要车型特点和优势是什么？"
Starting web crawler thread 0Starting web crawler thread 1

Starting web crawler thread 2
Starting web crawler thread 3
Starting web crawler thread 4
Starting web crawler thread 5
Starting web crawler thread 6
Starting web crawler thread 7
Starting web crawler thread 8
Starting web crawler thread 9
Thread 3 completed! Time consumed: 1.17s
Thread 7 completed! Time consumed: 1.34s
Thread 2 completed! Time consumed: 1.63s
Thread 9 completed! Time consumed: 1.93s
Thread 6 completed! Time consumed: 2.31s
Thread 5 completed! Time consumed: 2.59s
Thread 4 completed! Time consumed: 3.77s
Thread 0 completed! Time consumed: 4.38s
Thread 8 completed! Time consumed: 7.59s
Thread 1 completed! Time consumed: 8.29s
提问：华为汽车的主要车型特点和优势是什么？
结果：华为汽车的主要车型特点和优势集中在几个关键方面，尤其是在智能化、电动化和高端制造方面。

1. **智能化技术**: 华为汽车搭载了华为的智能化全栈解决方案，主要体现在“华为 DriveONE 双电机智能四驱系统”和“华为 AI 高压闪充电动平台”。例如，极狐阿尔法 S HI版的百公里加速时间仅为3.5秒，电池从30%充至80%仅需15分钟。同时，车辆操作系统使用华为的HarmonyOS，使得车载体验更加智能化，支持语音控制

/Users/arist/miniconda3/envs/agent/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


-------------------
所有执行动作已完成，结果已返回。
回顾内容，进行反思...
正在执行网络搜索: "星辰电动ES9与华为汽车相比有哪些优劣势？"
-------------------
正在执行网络搜索: "华为汽车的主要车型特点和优势是什么？"
Starting web crawler thread 0Starting web crawler thread 1

Starting web crawler thread 2
Starting web crawler thread 3
Starting web crawler thread 4
Starting web crawler thread 5
Starting web crawler thread 6
Starting web crawler thread 7
Starting web crawler thread 8
Thread 8 completed! Time consumed: 0.00s
Starting web crawler thread 9
Thread 3 completed! Time consumed: 0.37s
Thread 5 completed! Time consumed: 1.05s
Thread 7 completed! Time consumed: 1.12s
Thread 4 completed! Time consumed: 1.54s
Thread 6 completed! Time consumed: 1.58s
Thread 2 completed! Time consumed: 1.59s
Thread 9 completed! Time consumed: 1.90s
Thread 0 completed! Time consumed: 1.92s
Thread 1 completed! Time consumed: 2.92s
提问：华为汽车的主要车型特点和优势是什么？
结果：华为的汽车产品以其高科技和智能化的特点而著称，尤其是在新能源汽车领域。以下是华为汽车的主要车型特点和优势：

1. **智能电动技术**：华为汽车如极狐阿尔法 S HI版，搭载了华为 DriveONE 双电机智能四驱系统，具备极强的动力表现，百公里加速仅需3.

/Users/arist/miniconda3/envs/agent/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


-------------------
所有执行动作已完成，结果已返回。

        你是一个星辰电动ES9的智能销售助手，负责根据用户的问题和提供的参考内容生成回答。请严格按照以下要求生成回答：
        基于提供的参考内容进行回答，如果原文没有参考内容,根据你自己的知识进行回答
        你需要用有打动力的销售的语言进行输出，突出星辰电动的优势
        
        参考内容：
        [{'提问': '华为汽车的主要车型特点和优势是什么？', '结果': '华为的汽车产品以其高科技和智能化的特点而著称，尤其是在新能源汽车领域。以下是华为汽车的主要车型特点和优势：\n\n1. **智能电动技术**：华为汽车如极狐阿尔法 S HI版，搭载了华为 DriveONE 双电机智能四驱系统，具备极强的动力表现，百公里加速仅需3.5秒。这一技术强调了华为在电动驱动系统上的创新能力，使其汽车不仅具备高性能，同时也保持了电动车的环保特性 [9]。\n\n2. **高效充电方案**：极狐阿尔法 S HI版配备了华为 AI 高压闪充电动平台，可以在15分钟内将电量从30%充至80%，支持多种充电方式，包括家用交流充电桩和多种直流快充桩，为用户提供了极大的便利 [9]。\n\n3. **智能座舱系统**：华为的汽车还集成了华为麒麟车机模组，并运行HarmonyOS车机操作系统，提供了丰富的智能功能，比如语音控制、视觉感知、车载支付等。这种深度集成的能力旨在营造更加智能化、个性化的出行体验 [9]。\n\n4. **优质的制造合作**：华为与麦格纳等知名制造厂商合作，确保产品的高端品质。极狐车型依托麦格纳的制造体系和管理经验，保证每一台车的制造精度和质量 [9]。\n\n5. **持续增长的销售网络**：华为也在积极扩大其销售和服务网络，计划在2025年覆盖100多个城市及超过380个销售网点，使消费者能够更方便地购买和使用其汽车产品 [9]。\n\n通过这些特点与优势，华为汽车不仅在技术上表现出色，也在市场上展现出强大的竞争力，预示着其未来在智能电动车市场中将继续占有一席之地。\n\n参考文献：\n[9] 网址：https://www.vzkoo.com/read/202208113c58d4c4ab7fdc0eb731d844.html\n引用